In [3]:
import pandas as pd
import numpy as np

class TradingStrategy:
    def __init__(self, build_threshold, liquidate_threshold):
        self.build_threshold = build_threshold
        self.liquidate_threshold = liquidate_threshold
        self.position = 0

    def generate_positions(self, data):
        data['position'] = 0
        for i in range(1, len(data)):
            alpha = data.loc[i, 'alpha']
            if self.position == 0:
                if alpha >= self.build_threshold:
                    self.position = 1
                elif alpha <= -self.build_threshold:
                    self.position = -1
            elif self.position == 1:
                if alpha <= self.liquidate_threshold:
                    self.position = 0
            elif self.position == -1:
                if alpha >= -self.liquidate_threshold:
                    self.position = 0
            data.loc[i, 'position'] = self.position
        return data

class BacktestingEngine:
    def __init__(self, data):
        self.data = data

    def calculate_pnl(self, strategy_data):
        strategy_data['return'] = strategy_data['price'].pct_change().fillna(0)
        strategy_data['PnL'] = strategy_data['position'].shift(1) * strategy_data['return']
        strategy_data['PnL'] = strategy_data['PnL'].fillna(0)
        strategy_data['cumulative_PnL'] = strategy_data['PnL'].cumsum()
        return strategy_data

class ThresholdOptimizer:
    def __init__(self, data):
        self.data = data

    def optimize(self, build_thresholds, liquidate_thresholds):
        best_PnL = -np.inf
        best_thresholds = (0, 0)
        for build_threshold in build_thresholds:
            for liquidate_threshold in liquidate_thresholds:
                strategy = TradingStrategy(build_threshold, liquidate_threshold)
                strategy_data = strategy.generate_positions(self.data.copy())
                engine = BacktestingEngine(strategy_data)
                result_data = engine.calculate_pnl(strategy_data)
                cumulative_PnL = result_data['cumulative_PnL'].iloc[-1]
                if cumulative_PnL > best_PnL:
                    best_PnL = cumulative_PnL
                    best_thresholds = (build_threshold, liquidate_threshold)
        return best_thresholds, best_PnL

# Load the dataset
data = pd.read_csv('asset_1.csv')


build_thresholds = np.arange(0.1, 1.0, 0.1)
liquidate_thresholds = np.arange(0.1, 1.0, 0.1)


optimizer = ThresholdOptimizer(data)


best_thresholds, best_PnL = optimizer.optimize(build_thresholds, liquidate_thresholds)


print(f'Best thresholds: Build = {best_thresholds[0]}, Liquidate = {best_thresholds[1]}')
print(f'Best PnL: {best_PnL}')


Best thresholds: Build = 0.30000000000000004, Liquidate = 0.1
Best PnL: 2.0905118327390944
